<a href="https://colab.research.google.com/github/sallywang147/SCInvarinfer/blob/main/Testcase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install --quiet accelerate
!pip install -i --quiet https://test.pypi.org/simple/ bitsandbytes
!pip install --quiet sentencepiece
!pip install --quiet transformers
!pip install --quiet rich[jupyter]
!pip install --quiet deepspeed
!pip install --quiet openai
!pip install --quiet gradio
!pip install --quiet peft
!pip install --quiet --upgrade datasets
!pip install --quiet simplejson
!pip install --quiet accelerate
!pip install --quiet bitsandbytes

Looking in indexes: --quiet
  Using cached https://test.pypi.org/simple/ (9.7 MB)
  ERROR: Cannot unpack file /tmp/pip-unpack-5cb7qpz7/simple.html (downloaded from /tmp/pip-req-build-gs15a5nj, content-type: text/html); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-gs15a5nj


In [9]:
import os
import torch
import sys, time
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

##Precise CoT-style Prompts##

In [15]:
context = "these are two relevant functions in a smart contract: function incremental_counter(uint counter) internal returns(uint) { counter +=1; return counter; } function manipulate_hash(bytes32 val) public returns (bytes32) { string memory mytest = 'abc'; bytes32 val2 = keccak256(abi.encodePacked((mytest))); if (val > val2){ return val2; } return val; } function restrictive_branch(bytes32 val, bytes memory test_string) public returns (int){ uint x = 3; uint y = 5; byte one_byte; uint counter = 0; string memory val2 = 'hello world'; uint sum = x + y; bytes32 new_val = manipulate_hash(val); counter = incremental_counter(counter); bytes32 output = keccak256(abi.encodePacked(new_val, keccak256('MY_SALT'))); bytes32 out_val = keccak256(abi.encodePacked(val, keccak256(test_string))); //to pass if, we need: //i)test_string = 'MY_SALT' and ii) val <=keccak256(abi.encodePacked((mytest)) if (out_val==output){ //bytes memory join_string = abi.encodePacked(val, val2); if(keccak256('23') < keccak256(abi.encodePacked(block.difficulty, block.timestamp, counter))){ one_byte = output[1]; }else{ one_byte = output[2]; } uint divide = ((sum * 3)**uint(4))/5; uint mod = divide%33; return int(mod); } return int(sum); }"
coverage_gap = "Due to restrictive if condition if (out_val==output), here are unexecuted lines during fuzzing: //bytes memory join_string = abi.encodePacked(val, val2); if(keccak256('23') < keccak256(abi.encodePacked(block.difficulty, block.timestamp, counter))){ one_byte = output[1]; }else{ one_byte = output[2]; } uint divide = ((sum * 3)**uint(4))/5; uint mod = divide%33; return int(mod);"
bridge_gap = "Please find valid out_val and output that can pass the previously given if condition if (out_val==output)"
reinforce_value = "Please remember the output of varaible out_val and the output of variable output from the previous prompt for the next prompt"
testcase_gen = "Given this json template [{'call':{'contents':['restrictive_branch', [{'contents':[32,'blank_1'],'tag':'AbiBytes'},{'contents':'blank_2','tag':'AbiBytesDynamic'}]],'tag':'SolCall'},'delay':['0x00000000000000000000000000000000000000000000000000000000000142ee','0x0000000000000000000000000000000000000000000000000000000000003032'],'dst':'0x00a329c0648769A73afAc7F9381E08FB43dBEA72','gas':12500000,'gasprice':'0x0000000000000000000000000000000000000000000000000000000000000000','src':'0x0000000000000000000000000000000000010000','value':'0x0000000000000000000000000000000000000000000000000000000000000000'}]. Please replace blank_1 with the output of variable out_val and replace blank_2 with the output of variable output in previous step. Then please generate a json file with replaced values."

In [16]:
prompt =[]
prompt.append(context)
prompt.append(coverage_gap)
prompt.append(bridge_gap)
prompt.append(reinforce_value)
prompt.append(testcase_gen)

##Falcon-7B##

Too slow, can't do math well

In [6]:
pip install --quiet einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.2 MB/s eta 0:00:00


In [7]:
model_name = "tiiuae/falcon-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#Note: falcon is terribly slow. On this simple promt, it took about 3 min
simple_prompt = "Please compute the value of keccak256(abi.encodePacked(\"hello world\"))"
input = tokenizer.encode(simple_prompt, return_tensors="pt")
output = model.generate(input, max_length=500,
                            do_sample=True,
                            top_p=0.95,
                            top_k=50,
                            temperature=0.8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id)
print(tokenizer.decode(output[0]))

In [12]:
falcon_start = time.time()
for i in range(len(prompt)):
    input = tokenizer.encode(prompt[i], return_tensors="pt")
    output = model.generate(input, max_length=500,
                            do_sample=True,
                            top_p=0.95,
                            top_k=50,
                            temperature=0.8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id)
    print(tokenizer.decode(output[0]))
falcon_end = time.time()
falcon_runtime = falcon_end - falcon_start
print('Falcon inference takes {:0.2f}s'.format(falcon_runtime))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Input length of input_ids is 428, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Input length of input_ids is 137, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


these are two relevant functions in a smart contract: function incremental_counter(uint counter) internal returns(uint) { counter +=1; return counter; } function manipulate_hash(bytes32 val) public returns (bytes32) { string memory mytest = 'abc'; bytes32 val2 = keccak256(abi.encodePacked((mytest))); if (val > val2){ return val2; } return val; } function restrictive_branch(bytes32 val, bytes memory test_string) public returns (int){ uint x = 3; uint y = 5; byte one_byte; uint counter = 0; string memory val2 = 'hello world'; uint sum = x + y; bytes32 new_val = manipulate_hash(val); counter = incremental_counter(counter); bytes32 output = keccak256(abi.encodePacked(new_val, keccak256('MY_SALT'))); bytes32 out_val = keccak256(abi.encodePacked(val, keccak256(test_string))); //to pass if, we need: //i)test_string = 'MY_SALT' and ii) val <=keccak256(abi.encodePacked((mytest)) if (out_val==output){ //bytes memory join_string = abi.encodePacked(val, val2); if(keccak256('23') < keccak256(abi.en

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Due to restrictive if condition if (out_val==output), here are unexecuted lines during fuzzing: //bytes memory join_string = abi.encodePacked(val, val2); if(keccak256('23') < keccak256(abi.encodePacked(block.difficulty, block.timestamp, counter))){ one_byte = output[1]; }else{ one_byte = output[2]; } uint divide = ((sum * 3)**uint(4))/5; uint mod = divide%33; return int(mod); 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Please find valid out_val and output that can pass the previously given if condition if (out_val==output) and (value_out_valid==value)

and I would like to know the output of out_val and output in case when the if condition fails (out_val is not equal to output).
>>COMMENT<< What is the expected output?>>COMMENT<< I would like to know the output of out_val and output in case when the if condition fails 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Input length of input_ids is 261, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Please remember the output of varaible out_val and the output of variable output from the previous prompt for the next prompt.
2.2.1
Write a C program that accepts two numbers as input. It should print the smaller of the two numbers on the standard output.
2.2.2
Write a C program that accepts two numbers as input. It should print the smaller of the two numbers on the standard output.
2.2.3
Write a C program
Given this json template [{'call':{'contents':['restrictive_branch', [{'contents':[32,'blank_1'],'tag':'AbiBytes'},{'contents':'blank_2','tag':'AbiBytesDynamic'}]],'tag':'SolCall'},'delay':['0x00000000000000000000000000000000000000000000000000000000000142ee','0x0000000000000000000000000000000000000000000000000000000000003032'],'dst':'0x00a329c0648769A73afAc7F9381E08FB43dBEA72','gas':12500000,'gasprice':'0x0000000000000000000000000000000000000000000000000000000000000000','src':'0x0000000000000000000000000000000000010000','value':'0x0000000000000000000000000000000000000000000000000000

##StarCoder##
Note: since their repo is private, I conducted inference on their huggingface API instead

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigcode/starcoder"
device = "cuda" # for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

for i in range(len(prompt)):
    input = tokenizer.encode(prompt[i], return_tensors="pt").to(device)
    output = model.generate(input, max_length=100, do_sample=True, top_p=0.95, top_k=50, temperature=0.8, repeat_penalty=0.2)
    print(tokenizer.decode(output[0]))


##Wizard Coder##

In [ ]:
!pip install jsonlines
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint ="WizardLM/WizardCoder-15B-V1.0"  #"bigcode/starcoder"
device = "cuda" # for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

for i in range(len(prompt)):
    input = tokenizer.encode(prompt[i], return_tensors="pt").to(device)
    output = model.generate(input, max_length=100, do_sample=True, top_p=0.95, top_k=50, temperature=0.8, repeat_penalty=0.2)
    print(tokenizer.decode(output[0]))


##Bard##

 It's really the **best for few-shot prompting**

In [ ]:
!pip install --quiet bardapi
import os
import requests
from bardapi import Bard

os.environ['_BARD_API_KEY']='YggjmkEptP7-cTTLlfnuWd5o2KvaRYr66kF6DbY6lvWf18tszK1IIq3DHnxP81IVjYkPlw.'
session = requests.Session()
session.headers = {
            "Host": "bard.google.com",
            "X-Same-Domain": "1",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
            "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8",
            "Origin": "https://bard.google.com",
            "Referer": "https://bard.google.com/",
        }
session.cookies.set("__Secure-1PSID", os.environ["_BARD_API_KEY"])

In [18]:
bard = Bard(session=session)
simple_prompt = "Please compute the value of keccak256(abi.encodePacked(\"hello world\"))"
print(bard.get_answer(simple_prompt)['content'])

bard_start = time.time()
for i in range(len(prompt)):
    print(bard.get_answer(prompt[i])['content'])
bard_end = time.time()
bard_runtime = bard_end - bard_start
print('Bard inference takes {:0.2f}s'.format(bard_runtime))


Sure, the value of keccak256(abi.encodePacked("hello world")) is **0x47173285a8d7341e5e972fc677286384f802f8ef42a5ec5f03bbfa254cb01fad**.

The abi.encodePacked function takes a string as input and returns a packed encoding of the string. The packed encoding is a byte array that contains the bytes of the string, in order.

The keccak256 function takes a byte array as input and returns a 256-bit hash of the byte array.

In this case, the abi.encodePacked function will return a packed encoding of the string "hello world". The keccak256 function will then take the packed encoding as input and return a 256-bit hash of the packed encoding.

The 256-bit hash of the packed encoding of the string "hello world" is **0x47173285a8d7341e5e972fc677286384f802f8ef42a5ec5f03bbfa254cb01fad**.

You can verify this value by using a hashing library or online tool.
Sure, I can help you understand these functions.

The first function, `incremental_counter`, increments the value of the `counter` variable by 1.

##LLaMA-7B##

In [12]:
#!pip install --quiet --upgrade git+https://github.com/huggingface/transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, LlamaForCausalLM

llama_tokenizer = LlamaTokenizer.from_pretrained(
            "decapoda-research/llama-7b-hf")

llama = LlamaForCausalLM.from_pretrained(
            "decapoda-research/llama-7b-hf")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
simple_prompt = "Please compute the value of keccak256(abi.encodePacked(\"hello world\"))"
input = llama_tokenizer.encode(simple_prompt, return_tensors="pt")
output = llama.generate(input, max_length=500,
                            do_sample=True,
                            top_p=0.95,
                            top_k=50,
                            temperature=0.8)
print(llama_tokenizer.decode(output[0]))

llama_start = time.time()
for i in range(len(prompt)):
    input = llama_tokenizer.encode(prompt[i], return_tensors="pt")
    output = llama.generate(input, max_length=100, do_sample=True, top_p=0.95, top_k=50, temperature=0.8, repeat_penalty=0.2)
    print(llama_tokenizer.decode(output[0]))
    '''
    llama_tokenizer.batch_decode(output,
                       skip_special_tokens=True,
                       temperature=0.8,
                       top_p=0.95,
                       top_k=50,
                       repeat_penalty=0.2,
                       clean_up_tokenization_spaces=False)[0]
    print(llama_tokenizer.decode(output[0])
    '''
llama_end = time.time()
llama_runtime = llama_end - llama_start
print('Llama inference takes {:0.2f}s'.format(llama_runtime))